# A bit on performance: `sum`

The  **sum** function `sum(a)`, which computes

$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i.
$$

In [ ]:
#Pkg.add("BenchmarkTools")
using BenchmarkTools

# 1a. serial Julia (as fast as handwritten c)

In [ ]:
function jsum(A)   
    s = 0.  # s = zero(eltype(A))
    for a in A
        s += a
    end
    s
end

In [ ]:
a = randn(10^7) # array of random numbers, uniform on [0,1)
j_bench = @benchmark jsum($a)

In [ ]:
minimum(j_bench.times)/1e6

## 1b. From the standard library --- but also Julia!

In [ ]:
sum_bench = @benchmark sum($a)
minimum(sum_bench.times)/1e6

# 2a. Distributed Computing with DArrays

This provides a very high level of abstraction of parallelism, where you just do whole-array operations and all the parallelism and data distribution is hidden.

At this time, this package is not as complete as it should be.

In [ ]:
workers()

In [ ]:
Pkg.add("DistributedArrays")
using DistributedArrays

In [ ]:
A = distribute(a)  # one could have used A = drand(10^7) for a parallel constructor
A.indexes

In [ ]:
addprocs(2)

In [ ]:
workers() # 2 and 3 are workers  (so in parallel we have two processors)

In [ ]:
@everywhere using DistributedArrays
A = distribute(a)
A.indexes

In [ ]:
sum(A)-sum(a)

In [ ]:
minimum((@benchmark sum(erf,$A)).times)/1e6

In [ ]:
minimum((@benchmark sum(erf,$a)).times)/1e6

## 2b. How do we implement DArray functionality?  Answer: Bootstrap from serial functionality.

In [ ]:
A = drand(10^6)

In [ ]:
a = convert( Array{Float64,1} , A)

In [ ]:
# Simulated Parallel Algorithm
s = sum.([a[i[1]] for i in A.indexes]) 
display(s)
sum(s)

In [ ]:
# Actual Parallel Algorithm

In [ ]:
for p in A.pids
    display(fetch(@spawnat p sum(localpart(A))))
end

In [ ]:
results = Float64[]
@sync for p in A.pids
     @async push!(results, fetch(@spawnat p sum(localpart(A))))
end

println(results)
sum(results)

# 2c. Constructing DArrays from pieces

In [ ]:
d = DArray((10,10)) do I
    println("processor $(myid()) is handling indices $I")
    fill(myid(), map(length, I))
end

#### Exercise

Write a function to transpose a distributed square matrix:

```
function transpose{T}(d::DArray{T,2})
    DArray((size(d,2),size(d,1))) do I
        ...
    end
end
```

# 2d. DArrays of anything

In [ ]:
D55 = @DArray [ randn(5,5) for i = 1:32]

In [ ]:
Dsvd = map(svdvals, D55)

## 3. @parallel  (looks like a "for loop", data starts and ends on master)

The history of Julia includes a "back and forth" between "for loop" notation and vectorized
notation.  The truth is, for some problems, one or the other, or both could be more natural,
or more performant, but not for the old reasons of dynamic languages where "for loops" were
always to be avoided.

In [ ]:
@everywhere randstat(n) = maximum(eigvals(SymTridiagonal(randn(n),randn(n-1))))

In [ ]:
@parallel (+) for i in 1:250
    randstat(500) 
end

In [ ]:
s = map(randstat, dfill(500,2000))

In [ ]:
using Plots
gr()

In [ ]:
histogram(s)

## 4. MPI is possible for the die-hards and also some libraries

(See separate notebook)

## 5. Threads under development

In [ ]:
function sim(n, seed = rand(UInt))
    nt = Threads.nthreads()
    ra = zeros(nt)
    Threads.@threads for i in 1:nt
        id   = Threads.threadid()
        rng  = MersenneTwister(id + seed)
        d, r = divrem(n, Threads.nthreads())
        bs = d + (r > 0)
        if id == nt
            bs = n - (nt - 1)*bs
        end
        ra[id] = sum(rand(rng, bs))
    end
    return sum(ra)
end

In [ ]:
@benchmark sim(10^7)

In [ ]:
function sim2(a)
    nt = Threads.nthreads()
    return sum(a)
end

In [ ]:
a =  rand(10^7)
@benchmark sim2(a)

## 6.  GPU Libraries using ArrayFire

Step 1. Follow directions in https://github.com/JuliaComputing/ArrayFire.jl from a terminal to load arrayfire on your machine.

In [ ]:
using ArrayFire

In [ ]:
getActiveBackend()

In [ ]:
getAvailableBackends()